5 DAY DATA RETRIEVAL

In [ ]:
import yfinance as yf
import pandas as pd
import os

# Define the stock ticker symbol
ticker_symbol = 'WFRD'

# Define the fixed date range from 12/03/2024 to 12/10/2024
start_date = '2024-12-06'
end_date = '2024-12-13'

# Step 1: Print the date range
print(f"Fetching data from {start_date} to {end_date}...")

# Step 2: Retrieve intraday data with a 60-minute interval
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, interval="60m")

# Check if data was retrieved
if stock_data.empty:
    raise ValueError("No data retrieved. Check the ticker or internet connection.")

# Step 3: Calculate VWAP
stock_data['Typical Price'] = (stock_data['High'] + stock_data['Low'] + stock_data['Close']) / 3
stock_data['Cumulative TP*Volume'] = (stock_data['Typical Price'] * stock_data['Volume']).cumsum()
stock_data['Cumulative Volume'] = stock_data['Volume'].cumsum()
stock_data['VWAP'] = stock_data['Cumulative TP*Volume'] / stock_data['Cumulative Volume']

# Step 4: Calculate Bollinger Bands
rolling_window = 20  # Rolling window for Bollinger Bands
stock_data['Rolling Mean'] = stock_data['Close'].rolling(window=rolling_window).mean()
stock_data['Rolling Std'] = stock_data['Close'].rolling(window=rolling_window).std()
stock_data['Upper Band'] = stock_data['Rolling Mean'] + (2 * stock_data['Rolling Std'])
stock_data['Lower Band'] = stock_data['Rolling Mean'] - (2 * stock_data['Rolling Std'])

# Step 5: Select the desired columns
final_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'Rolling Mean', 'Upper Band', 'Lower Band']
final_data = stock_data[final_columns]

# Step 6: Save the data to the Desktop as a CSV file
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
csv_file_path = os.path.join(desktop_path, f"{ticker_symbol}_intraday_12_06_to_12_12_with_BollingerBands_and_VWAP.csv")
final_data.to_csv(csv_file_path, index=True)

print(f"Data saved to {csv_file_path}")


VISUAL DISPLAY

In [ ]:
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import os

# Path to the CSV file on the Desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
csv_file_path = os.path.join(desktop_path, 'WFRD_intraday_12_06_to_12_12_with_BollingerBands_and_VWAP.csv')  # The file we just saved

# Load your data
data = pd.read_csv(csv_file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Set the 'Datetime' as the index for easier time-based manipulation
data.set_index('Datetime', inplace=True)

# Calculate VWAP
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Calculate Bollinger Bands
data['BB_Middle'] = data['Close'].rolling(window=20).mean()
data['BB_Upper'] = data['BB_Middle'] + 2 * data['Close'].rolling(window=20).std()
data['BB_Lower'] = data['BB_Middle'] - 2 * data['Close'].rolling(window=20).std()

# Filter the data for visualization if needed
start_date = data.index.min().strftime('%Y-%m-%d')  # Extract the start date
end_date = data.index.max().strftime('%Y-%m-%d')    # Extract the end date
filtered_data = data[(data.index >= data.index.min()) & (data.index <= data.index.max())]

# Create a custom plot of the candlestick chart with VWAP and Bollinger Bands
ap = [
    mpf.make_addplot(filtered_data['VWAP'], color='orange', linestyle='-', label='VWAP'),
    mpf.make_addplot(filtered_data['BB_Upper'], color='green', linestyle='--', label='Bollinger Upper'),
    mpf.make_addplot(filtered_data['BB_Lower'], color='red', linestyle='--', label='Bollinger Lower'),
    mpf.make_addplot(filtered_data['BB_Middle'], color='blue', linestyle='-', label='Bollinger Middle')
]

# Generate the candlestick chart with volume, VWAP, and Bollinger Bands
pdf_file_path = os.path.join(desktop_path, f'WFRD_intraday_candlestick_{start_date}_to_{end_date}.pdf')
mpf.plot(
    filtered_data,
    type='candle',
    volume=True,
    addplot=ap,
    title=f"WFRD Intraday Candlestick Chart ({start_date} to {end_date})",
    style='yahoo',
    savefig=pdf_file_path  # Save the figure as a PDF
)

print(f"Chart saved as a PDF at {pdf_file_path}")


5-DAY RETRIEVAL TO SPECIFIC SAVE PATH

In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta

# Define the file path and extract the date
csv_file_path = '/Volumes/RaymondSingh/Stocks/WFRD/2024-12-09/WFRD_intraday_2024-12-09_5-Day.csv'
date_str = csv_file_path.split('/')[-2]  # Extract "2024-06-05" from the file path
target_date = datetime.strptime(date_str, "%Y-%m-%d")

# Calculate the start and end dates for the 5-day interval
start_date = (target_date - timedelta(days=4)).strftime('%Y-%m-%d')  # 5 days, inclusive of the target date
end_date = (target_date + timedelta(days=1)).strftime('%Y-%m-%d')    # Extend end date to include the target date

# Step 1: Retrieve 5-day intraday data with a 60-minute interval
ticker_symbol = 'WFRD'
print(f"Retrieving 5-day intraday data for {ticker_symbol} from {start_date} to {end_date}...")
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, interval="60m")

# Check if data was retrieved
if stock_data.empty:
    raise ValueError("No data retrieved. Check the ticker or internet connection.")

# Step 2: Calculate VWAP
print("Calculating VWAP...")
stock_data['Typical Price'] = (stock_data['High'] + stock_data['Low'] + stock_data['Close']) / 3
stock_data['Cumulative TP*Volume'] = (stock_data['Typical Price'] * stock_data['Volume']).cumsum()
stock_data['Cumulative Volume'] = stock_data['Volume'].cumsum()
stock_data['VWAP'] = stock_data['Cumulative TP*Volume'] / stock_data['Cumulative Volume']

# Step 3: Calculate Bollinger Bands
print("Calculating Bollinger Bands...")
rolling_window = 20
stock_data['Rolling Mean'] = stock_data['Close'].rolling(window=rolling_window).mean()
stock_data['Rolling Std'] = stock_data['Close'].rolling(window=rolling_window).std()
stock_data['Upper Band'] = stock_data['Rolling Mean'] + (2 * stock_data['Rolling Std'])
stock_data['Lower Band'] = stock_data['Rolling Mean'] - (2 * stock_data['Rolling Std'])

# Step 4: Save to CSV
print(f"Saving data to {csv_file_path}...")
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
stock_data.to_csv(csv_file_path)

print(f"Data saved to {csv_file_path}")


5-Day VISUAL TO SPECIFIC PATH

In [ ]:
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import os

# Path to the specified CSV file
csv_file_path = '/Volumes/RaymondSingh/Stocks/WFRD/2024-12-09/WFRD_intraday_2024-12-09_5-Day.csv'

# Load your data
data = pd.read_csv(csv_file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

# Check dataset size
print(f"Number of rows in the dataset: {len(data)}")

# Calculate VWAP
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Adjust rolling window based on dataset size
window_size = min(20, len(data))
print(f"Using rolling window size: {window_size}")

# Calculate Bollinger Bands
data['BB_Middle'] = data['Close'].rolling(window=window_size).mean()
data['BB_Upper'] = data['BB_Middle'] + 2 * data['Close'].rolling(window=window_size).std()
data['BB_Lower'] = data['BB_Middle'] - 2 * data['Close'].rolling(window=window_size).std()

# Fill NaN values to avoid empty data
data.fillna(method='bfill', inplace=True)
print("NaN values handled. Dataset ready for plotting.")

# Filter the data for visualization
filtered_data = data.copy()
print(f"Filtered data rows: {len(filtered_data)}")

# Verify the data isn't empty
if filtered_data.empty:
    raise ValueError("Filtered data is empty. Ensure sufficient rows exist in the dataset.")

# Extract the start and end dates for the chart title
start_date = filtered_data.index.min().strftime('%Y-%m-%d')
end_date = filtered_data.index.max().strftime('%Y-%m-%d')

# Create a custom plot of the candlestick chart with VWAP and Bollinger Bands
ap = [
    mpf.make_addplot(filtered_data['VWAP'], color='orange', linestyle='-', label='VWAP'),
    mpf.make_addplot(filtered_data['BB_Upper'], color='green', linestyle='--', label='Bollinger Upper'),
    mpf.make_addplot(filtered_data['BB_Lower'], color='red', linestyle='--', label='Bollinger Lower'),
    mpf.make_addplot(filtered_data['BB_Middle'], color='blue', linestyle='-', label='Bollinger Middle')
]

# Path to save the PDF
pdf_file_path = '/Volumes/RaymondSingh/Stocks/WFRD/2024-12-09/WFRD_5_Day_candlestick_chart.pdf'

# Generate the candlestick chart
mpf.plot(
    filtered_data,
    type='candle',
    volume=True,
    addplot=ap,
    title=f"WFRD Intraday Candlestick Chart ({start_date} to {end_date})",
    style='yahoo',
    savefig=pdf_file_path
)

print(f"Chart saved as a PDF at {pdf_file_path}")
